## Bibliotecas

In [37]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score
from datetime import datetime
import NN
import nltk

## Leemos el excell para obtener el dataFrame

In [22]:
data = pd.read_excel("DatosSeguros_Eq.xlsx")

## Comenzamos a limpiar los datos de origen

In [23]:
data.head() #La cabecera del excel que hemos leído

ActualmenteAsegurado  ASinSiniestro  SiniestrosNoResponsable3YA  \
0                     1              5                           0   
1                     0              0                           0   
2                     1              1                           0   
3                     1              5                           0   
4                     0              0                           0   

   SiniestrosResponsable3YA  ConductoresAdicionalesConCarnet  \
0                         0                                0   
1                         0                                0   
2                         0                                0   
3                         0                                0   
4                         0                                0   

  FechaCarnetAdicional FechaNacimientoAdicional TipoCarnetAdicional  \
0                  NaT                      NaT                 NaN   
1                  NaT                      NaT                 NaN   
2                  NaT                      NaT                 NaN   
3                  NaT                      NaT                 NaN   
4                  NaT                      NaT                 NaN   

  ParentescoAdicional IDFiscalAdicional       ...        \
0                 NaN               NaN       ...         
1                 NaN               NaN       ...         
2                 NaN               NaN       ...         
3                 NaN               NaN       ...         
4                 NaN               NaN       ...         

          FechaRealizacion Estado Fecha de ocurrencia Fecha de apertura  \
0  2013-09-03 11:58:54.000      E                 NaN               NaN   
1  2016-01-21 08:47:51.000      E                 NaN               NaN   
2  2015-06-01 10:53:30.000      E                 NaN               NaN   
3  2017-07-26 19:55:59.000      K                 NaN               NaN   
4  2016-10-04 12:15:53.000      E                 NaN               NaN   

  Fecha de cierre  Tipo daño  Tipo siniestro  MatSin             IP  \
0             NaN        NaN             NaN     NaN            NaN   
1             NaN        NaN             NaN     NaN  80.30.252.181   
2             NaN        NaN             NaN     NaN  81.34.131.252   
3             NaN        NaN             NaN     NaN    37.14.13.21   
4             NaN        NaN             NaN     NaN  80.26.158.233   

  EsDeColaborador  
0               1  
1               1  
2               1  
3               0  
4               1  

[5 rows x 46 columns]

In [24]:
data.columns #Obtenemos las Columnas del excell

Index(['ActualmenteAsegurado', 'ASinSiniestro', 'SiniestrosNoResponsable3YA',
       'SiniestrosResponsable3YA', 'ConductoresAdicionalesConCarnet',
       'FechaCarnetAdicional', 'FechaNacimientoAdicional',
       'TipoCarnetAdicional', 'ParentescoAdicional', 'IDFiscalAdicional',
       'FechaVencimientoSeguro', 'Email', 'FechaCarnet', 'FechaNacimiento',
       'TipoCarnet', 'CPConduccion', 'Cilindrada', 'ConductorUnico',
       'LocalidadConduccion', 'Marca', 'Matricula', 'Modelo',
       'VehiculoHistorico', 'FechaMatriculacion', 'FechaAltaSolicitada',
       'AccidentesConductor', 'AsistenciaEnViaje', 'AsistenciaEnViajePremium',
       'AsistenciaEnViajePremium.1', 'Importe', 'ModalidadSeleccionada',
       'ClaseVehiculo', 'CodProhibicion', 'FechaLanzamiento', 'Cilindrada.1',
       'fechaefecto', 'FechaRealizacion', 'Estado', 'Fecha de ocurrencia',
       'Fecha de apertura', 'Fecha de cierre', 'Tipo daño', 'Tipo siniestro',
       'MatSin', 'IP', 'EsDeColaborador'],
      dtype='

In [25]:
clean_data = data.fillna(0)#Rellenamos con 0's los espacios en blanco/NULL que tenia nuestro documento de origen

In [26]:
#Eliminamos las columnas que no nos interesan.
dropable = ['ActualmenteAsegurado','SiniestrosNoResponsable3YA','SiniestrosResponsable3YA','ConductoresAdicionalesConCarnet','FechaCarnetAdicional','FechaNacimientoAdicional','TipoCarnetAdicional','ParentescoAdicional','IDFiscalAdicional','FechaVencimientoSeguro','LocalidadConduccion','Matricula','FechaAltaSolicitada','AsistenciaEnViaje','CodProhibicion','FechaLanzamiento','fechaefecto','FechaRealizacion','Estado','Fecha de ocurrencia','Fecha de ocurrencia','Fecha de apertura','Fecha de cierre','Tipo daño','MatSin','IP','EsDeColaborador','AsistenciaEnViajePremium']
clean_data = clean_data.drop(dropable, axis = 1) #Axis=1 para que sepa que es una columna

clean_data.columns
#Mostramos como queda con los features que sí nos interesa
#clean_data.head(10)

Index(['ASinSiniestro', 'Email', 'FechaCarnet', 'FechaNacimiento',
       'TipoCarnet', 'CPConduccion', 'Cilindrada', 'ConductorUnico', 'Marca',
       'Modelo', 'VehiculoHistorico', 'FechaMatriculacion',
       'AccidentesConductor', 'AsistenciaEnViajePremium.1', 'Importe',
       'ModalidadSeleccionada', 'ClaseVehiculo', 'Cilindrada.1',
       'Tipo siniestro'],
      dtype='object')

In [27]:
#Limpiamos los datos convirtiendo un string en enteros según nos interese y poniendo el nombre que nos interese a la columna.
clean_data['Tipo de Pago'] = clean_data['AsistenciaEnViajePremium.1'].map({'PagoAnual':1, 'PagoSemestral':2, 'PagoTrimestral':3}).astype(int)
clean_data['ModalidadSeleccionada'] = clean_data['ModalidadSeleccionada'].map({'ModalidadBasica':1, 'ModalidadAmpliada':2, 'TodoRiesgoConFranquicia':3}).astype(int)
#Una vez que ya hemos utilizado la columna a nuestro gusto, ya no la necesitamos y la eliminamos.
clean_data = clean_data.drop(['AsistenciaEnViajePremium.1'], axis = 1)

#Mostramos como queda con los features que sí nos interesa
clean_data.head(20)

ASinSiniestro        Email FechaCarnet FechaNacimiento TipoCarnet  \
0               5  hotmail.com  2000-10-31      1981-10-16          A   
1               0    gmail.com  2009-09-16      1991-07-22          B   
2               1  hotmail.com  2008-05-07      1989-04-11          B   
3               5  hotmail.com  2016-06-13      1979-09-24         A2   
4               0    gmail.com  2001-10-29      1982-06-26          B   
5               5  hotmail.com  1997-11-30      1970-01-20         Li   
6               0  hotmail.com  1984-06-30      1956-03-01          A   
7               5  hotmail.com  2012-01-31      1990-03-20          B   
8               4  HOTMAIL.COM  2013-11-25      1991-03-16          A   
9               5   HOTMAIL.ES  1990-11-30      1972-11-21          B   
10              4    gmail.com  2007-02-28      1981-01-14          B   
11              5  hotmail.com  2016-09-08      1972-12-09          A   
12              5   hotmail.es  2002-11-30      1982-03-22          A   
13              4  hotmail.com  1997-06-30      1977-07-27          A   
14              5    gmail.com  1999-06-30      1980-08-31          A   
15              4    gmail.com  2007-07-16      1987-01-25          A   
16              5  hotmail.com  1995-04-30      1966-12-20          B   
17              5    gmail.com  2004-04-22      1985-11-30          B   
18              5    gmail.com  1977-12-20      1958-04-08          A   
19              0  hotmail.com  2013-05-24      1993-07-10         AM   

    CPConduccion  Cilindrada  ConductorUnico          Marca           Modelo  \
0          13170         599               1         YAMAHA          YZF 600   
1          28016         124               1         DAELIM          ROADWIN   
2          28031         125               1         KEEWAY       SUPERLIGHT   
3           8921         750               1        APRILIA        SL SHIVER   
4           3001         125               1        HYOSUNG              MS3   
5           8380          49               0          DERBI            SENDA   
6          17320         125               1            KTM          EXC 125   
7           7010          49               1         YAMAHA           JOG 50   
8           8031         748               1       KAWASAKI            Z 750   
9          30205         653               1     BOMBARDIER           DS 650   
10         28026         125               1         SAMADA             POLE   
11          8940        1148               1            KTM              RC8   
12         11130         600               1         YAMAHA              FZ6   
13          8980         499               1         YAMAHA           XP 500   
14         48180         675               1        Triumph    Street Triple   
15         27002         599               1         YAMAHA          YZF 600   
16         17246         659               1         YAMAHA        YFM 660 R   
17         14113         125               1        APRILIA       FUTURA 125   
18         18230         249               1          KYMCO   GRAND DINK 250   
19         35016          49               1  PIAGGIO-VESPA  NRG 50 (ENERGY)   

    VehiculoHistorico FechaMatriculacion  AccidentesConductor  Importe  \
0                   0         2000-09-03                    0   144.33   
1                   0         2010-01-21                    0   157.83   
2                   0         2013-10-11                    0   130.29   
3                   0         2010-07-26                    0   247.05   
4                   0         2007-10-10                    0   116.92   
5                   0         2010-05-19                    1   343.54   
6                   0         2018-03-12                    0    86.39   
7                   0         2008-07-23                    0   247.72   
8                   0         2008-11-13                    1   390.33   
9                   0         2003-12-14         

In [28]:
clean_data.Email.unique()

array(['hotmail.com', 'gmail.com', 'HOTMAIL.COM', 'HOTMAIL.ES',
       'hotmail.es', 'msn.com', 'azet.sk', 'yahoo.es', 'grupobes.es',
       'GMAIL.COM', 'outlook.com', 'ono.com', 'pavonne.es', 'technal.com',
       'telefonica.net', 'rytabo.com', 'gmal.com', 'SALESFARMA.ES',
       'FUNERARIA-DELCARMEN.COM', 'druco.es', 'orange.es', 'enel.com',
       'iCloud.com', 'sabadellatlantico.com', 'outlook.es', 'gmx.es',
       'et.mde.es', 'EXTINTORESZENITN.COM', 'fide.es', 'mail.com',
       'abv.bg', 'military.airbus.com', 'cadigrafia.com',
       'lampisteriamauri.com', 'Gmail.com', 'CCMA.CAT', 'live.com',
       'luxuryspain.es', 'Hotmail.es', 'YAHOO.COM', 'seconsa.net',
       'afelsa.es', 'fpgestion.es', 'outlok.com', 'agricomex.es',
       'yahoo.com', 'libero.it', 'prepersa.es', 'inmoproin.com',
       'eresmas.com', 'Gestoriapastor.es', 'DANIELHIDALGO.ES',
       'gamail.es', 'web.de', 'comb.cat', 'Hotmail.com', 'me.com',
       'hotmail.commail.com', 'terra.com', 'regincos.com',
  

In [29]:

#Funcion para limpiar el tipo de siniestro segun nos convenga
def mapSiniestro():
    
    name = clean_data['Tipo siniestro'] #Cuando encontremos la columna TipoSiniestro del elemento iesimo
    
    #Cambiamos el valor segun nos interese
    
    clean_data.loc[name == 'CULPA', 'Tipo siniestro'] = 1
    clean_data.loc[name == 'RECLAMACIÓN', 'Tipo siniestro'] = 2


In [30]:
def mapMail():
    
    name = clean_data['Email']
    
    name = name.str.lower()
    
    clean_data.loc[name == 'hotmail.com', 'Email'] = 1
    clean_data.loc[name == 'hotmail.es', 'Email'] = 1
    clean_data.loc[name == 'gmail.com', 'Email'] = 2
    clean_data.loc[name == 'gmail.es', 'Email'] = 2
    clean_data.loc[name == 'yahoo.com', 'Email'] = 3
    clean_data.loc[name == 'yahoo.es', 'Email'] = 3

In [36]:
mail_int = 'hotmail.com'.astype(int)

AttributeError: 'str' object has no attribute 'astype'

In [34]:
clean_data.Email.unique()

array([0, 1, 'msn.com', 'azet.sk', 'yahoo.es', 'grupobes.es',
       'outlook.com', 'ono.com', 'pavonne.es', 'technal.com',
       'telefonica.net', 'rytabo.com', 'gmal.com', 'SALESFARMA.ES',
       'FUNERARIA-DELCARMEN.COM', 'druco.es', 'orange.es', 'enel.com',
       'iCloud.com', 'sabadellatlantico.com', 'outlook.es', 'gmx.es',
       'et.mde.es', 'EXTINTORESZENITN.COM', 'fide.es', 'mail.com',
       'abv.bg', 'military.airbus.com', 'cadigrafia.com',
       'lampisteriamauri.com', 'CCMA.CAT', 'live.com', 'luxuryspain.es',
       2, 'seconsa.net', 'afelsa.es', 'fpgestion.es', 'outlok.com',
       'agricomex.es', 'libero.it', 'prepersa.es', 'inmoproin.com',
       'eresmas.com', 'Gestoriapastor.es', 'DANIELHIDALGO.ES',
       'gamail.es', 'web.de', 'comb.cat', 'me.com', 'hotmail.commail.com',
       'terra.com', 'regincos.com', 'tresquartsdequinze.com', 'aena.es',
       'prosur.info', 'portsdebalears.com', 'mail.ru', 'hotmai.com',
       'CORREDORDESEGUROS.ES', 'zbl.es', 'marimon-abo

In [9]:
#Dá un futureWarning pero funciona, pero deberiamos quitarle.
mapSiniestro()

In [10]:
%matplotlib inline

#Sacamos el Año de las siguientes Fechas

clean_data['Año_Carnet'] = clean_data.FechaCarnet.dt.year.astype(int) 

clean_data['Año_Nacimiento']= clean_data.FechaNacimiento.dt.year.astype(int)  

clean_data['Año_Matriculacion']= clean_data.FechaMatriculacion.dt.year.astype(int)  

#Borramos las columnas una vez manipuladas.

clean_data = clean_data.drop(['FechaCarnet'], axis = 1)
clean_data = clean_data.drop(['FechaNacimiento'], axis = 1)
clean_data = clean_data.drop(['FechaMatriculacion'], axis = 1)

clean_data.head(10)

ASinSiniestro        Email TipoCarnet  CPConduccion  Cilindrada  \
0              5  hotmail.com          A         13170         599   
1              0    gmail.com          B         28016         124   
2              1  hotmail.com          B         28031         125   
3              5  hotmail.com         A2          8921         750   
4              0    gmail.com          B          3001         125   
5              5  hotmail.com         Li          8380          49   
6              0  hotmail.com          A         17320         125   
7              5  hotmail.com          B          7010          49   
8              4  HOTMAIL.COM          A          8031         748   
9              5   HOTMAIL.ES          B         30205         653   

   ConductorUnico       Marca      Modelo  VehiculoHistorico  \
0               1      YAMAHA     YZF 600                  0   
1               1      DAELIM     ROADWIN                  0   
2               1      KEEWAY  SUPERLIGHT                  0   
3               1     APRILIA   SL SHIVER                  0   
4               1     HYOSUNG         MS3                  0   
5               0       DERBI       SENDA                  0   
6               1         KTM     EXC 125                  0   
7               1      YAMAHA      JOG 50                  0   
8               1    KAWASAKI       Z 750                  0   
9               1  BOMBARDIER      DS 650                  0   

   AccidentesConductor  Importe  ModalidadSeleccionada ClaseVehiculo  \
0                    0   144.33                      1           M17   
1                    0   157.83                      1           M24   
2                    0   130.29                      1           M01   
3                    0   247.05                      1           M24   
4                    0   116.92                      1           M30   
5                    1   343.54                      1           C06   
6                    0    86.39                      1           M19   
7                    0   247.72                      1           C09   
8                    1   390.33                      1           M24   
9                    0   106.75                      1           M21   

   Cilindrada.1  Tipo siniestro  Tipo de Pago  Año_Carnet  Año_Nacimiento  \
0         599.0               0             1        2000            1981   
1         124.0               0             2        2009            1991   
2         125.0               0             1        2008            1989   
3         750.0               0             3        2016            1979   
4         125.0               0             1        2001            1982   
5          49.0               0             3        1997            1970   
6         125.0               0             1        1984            1956   
7          49.0               0             1        2012            1990   
8         748.0               0             1        2013            1991   
9         653.0               0             1        1990            1972   

   Año_Matriculacion  
0               2000  
1               2010  
2               2013  
3               2010  
4               2007  
5               2010  
6               2018  
7               2008  
8               2008  
9               2003

In [11]:
clean_data['Email'] = clean_data['Email'].str.lower() #Lo pasamos a minusculas


In [12]:
#Aplicamos un hash a las siguientes columnas para que la red neuronal pueda usar los datos
clean_data['Email'] = clean_data['Email'].apply(hash) 
clean_data['TipoCarnet'] = clean_data['TipoCarnet'].apply(hash)
clean_data['Marca'] = clean_data['Marca'].apply(hash)
clean_data['Modelo'] = clean_data['Modelo'].apply(hash)
clean_data['ClaseVehiculo'] = clean_data['ClaseVehiculo'].apply(hash)


## Red Neuronal

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

In [14]:
X = clean_data.drop("Tipo siniestro",1)
y = clean_data["Tipo siniestro"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    random_state=0)

In [16]:
clf = MLPClassifier()

In [17]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

X_train.head()

ASinSiniestro                Email           TipoCarnet  CPConduccion  \
15131              5 -6829451943348707713  5453146593686120554         15009   
20223              4  8674357812576261562  5453146593686120554          7320   
8595               5  4473465626418273524  5453146593686120554         33300   
9284               5 -6829451943348707713  -603165616239219985          8760   
19467              5 -6829451943348707713  8728238631488226975          8020   

       Cilindrada  ConductorUnico                Marca               Modelo  \
15131          49               1 -7577947541349744547  6750392013644249541   
20223         125               1 -1135703258337675330  1758305518082299293   
8595           49               1 -4756888226900955106  1908907022077838192   
9284          449               1 -9165090099688271963   249845000840611555   
19467          49               1 -1911733682614243818 -3221738536447942445   

       VehiculoHistorico  AccidentesConductor  Importe  ModalidadSeleccionada  \
15131                  0                    0   139.13                      1   
20223                  0                    1   238.50                      2   
8595                   1                    0    55.99                      1   
9284                   0                    0    84.80                      1   
19467                  0                    0   172.92                      1   

             ClaseVehiculo  Cilindrada.1  Tipo de Pago  Año_Carnet  \
15131   381249518735649731          49.0             1        2009   
20223  1268853694523307360         125.0             1        2007   
8595    381249518735649731          49.0             1        1993   
9284   4931072033683204031         449.0             1        1991   
19467                    0           0.0             1        2014   

       Año_Nacimiento  Año_Matriculacion  
15131            1972               2003  
20223            1989               2014  
8595             1970               1987  
9284             1972               2012  
19467            1980               2016

In [18]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [19]:
y_pred = clf.predict(X_test)
scores = cross_val_score(clf, X_test, y_test, cv=5)

In [20]:
scores

array([0.85344828, 0.85517241, 0.90517241, 0.88601036, 0.90484429])

In [21]:
confusion_matrix(y_test,y_pred)

array([[2566,  116,   11],
       [  76,    8,    1],
       [ 110,    9,    0]], dtype=int64)